
# Deep Neural Networks for Energy Forecasting


In this notebook, we will apply deep learning using PyTorch to a real-world energy use case:  
**Predicting backup generator activation for data centers based on environmental and operational variables**.

We will build and train several deep neural networks, compare activation functions, and evaluate model performance.

<h2>Libraries<h2>

<h5>Naturally, please install the below referenced libraries before attempting to run locally.<h5> 

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


## Step 1: Simulate Real-World Energy Data

In [ ]:

np.random.seed(0)

n_samples = 5000
X = pd.DataFrame({
    "grid_load_kw": np.random.normal(400, 100, n_samples),
    "fuel_pct_remaining": np.random.uniform(0.1, 1.0, n_samples),
    "temperature_c": np.random.normal(27, 5, n_samples),
    "humidity_pct": np.random.uniform(20, 90, n_samples),
    "wind_speed_kph": np.random.normal(15, 5, n_samples),
    "solar_irradiance_wm2": np.random.uniform(0, 1000, n_samples),
    "main_grid_stability_score": np.random.uniform(0.4, 1.0, n_samples),
    "minutes_since_last_start": np.random.randint(0, 180, n_samples)
})

# Binary target: generator starts when fuel is low, grid is unstable, or temp is high
y = ((X["fuel_pct_remaining"] < 0.3) & 
     (X["main_grid_stability_score"] < 0.6) & 
     (X["temperature_c"] > 30)).astype(int)

print(X.head())
print("Target distribution:", y.value_counts().to_dict())


## Step 2: Data Preprocessing

In [ ]:

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

input_dim = X_train.shape[1]


## Step 3: Define Deep Neural Network

In [ ]:

class EnergyNet(nn.Module):
    def __init__(self, input_dim, hidden1, hidden2, activation_fn):
        super(EnergyNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
        self.activation = activation_fn
        self.output = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.output(self.fc3(x))
        return x


## Step 4: Training Function

In [ ]:

def train_model(model, optimizer, criterion, X_train, y_train, X_test, y_test, epochs=200):
    loss_list, acc_list = [], []

    for epoch in range(epochs):
        model.train()
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_test_pred = model(X_test)
            preds = (y_test_pred > 0.5).float()
            acc = accuracy_score(y_test.numpy(), preds.numpy())

        loss_list.append(loss.item())
        acc_list.append(acc)

        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}, Accuracy = {acc:.4f}")

    return loss_list, acc_list


## Step 5: Compare Activation Functions

In [ ]:

results = {}
criterion = nn.BCELoss()
epochs = 200

activations = {
    "Sigmoid": torch.sigmoid,
    "Tanh": torch.tanh,
    "ReLU": torch.relu
}

for name, fn in activations.items():
    print(f"\nTraining with {name}")
    model = EnergyNet(input_dim, 32, 16, fn)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    loss, acc = train_model(model, optimizer, criterion, X_train, y_train, X_test, y_test, epochs)
    results[name] = {"loss": loss, "acc": acc}


## Step 6: Visualize Results

In [ ]:

plt.figure(figsize=(14,5))
for name in results:
    plt.plot(results[name]["loss"], label=f"{name} Loss")
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(14,5))
for name in results:
    plt.plot(results[name]["acc"], label=f"{name} Accuracy")
plt.title("Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()
